In [1]:
import pandas as pd
import html
import re
from bs4 import BeautifulSoup

def Process_COPOM_Comunicado(Comunicado):

   def Aux_COPOM_LimpaHTML(raw_html):
       if not raw_html:
           return "", None

       decoded = html.unescape(raw_html)
       decoded = re.sub(r'[\u200b\ufeff\u00a0]', '', decoded)
       soup = BeautifulSoup(decoded, 'html.parser')

       table_dfs = []
       for table in soup.find_all('table'):
           rows = []
           for tr in table.find_all('tr'):
               cells = [td.get_text(strip=True) for td in tr.find_all(['td', 'th'])]
               if cells:
                   rows.append(cells)
           if rows:
               df_table = pd.DataFrame(rows)
               table_dfs.append(df_table)
           table.decompose()

       for br in soup.find_all('br'):
           br.replace_with('\n')

       for p in soup.find_all('p'):
           p.insert_after('\n\n')

       text = soup.get_text()
       text = re.sub(r'\n\s*\n', '\n\n', text)  
       text = re.sub(r'[ \t]+', ' ', text)    
       text = text.strip()

       if table_dfs:
           return text, table_dfs[0]
       else:
           return text, None

   Link_Url = f"https://www.bcb.gov.br/api/servico/sitebcb/copom/comunicados_detalhes?nro_reuniao={Comunicado}"
   df = pd.read_json(Link_Url)
   Conteudo_Full = df.iloc[0]

   Reuniao_Raw = Conteudo_Full.iloc[0].get('nro_reuniao')
   DataReferencia_Raw = Conteudo_Full.iloc[0].get('dataReferencia')
   Titulo_Raw = Conteudo_Full.iloc[0].get('titulo')
   Comunicado_Raw = Conteudo_Full.iloc[0].get('textoComunicado')


   Comunicado_Texto, Comunicado_Tabela = Aux_COPOM_LimpaHTML(Comunicado_Raw)

   return Reuniao_Raw, DataReferencia_Raw, Titulo_Raw, Comunicado_Texto, Comunicado_Tabela


Reuniao_Raw, DataReferencia_Raw, Titulo_Raw, Comunicado_Texto, Comunicado_Tabela = Process_COPOM_Comunicado(Comunicado=274)
print(Reuniao_Raw, '\n\n\n', DataReferencia_Raw, '\n\n\n', Titulo_Raw, '\n\n\n', Comunicado_Texto)
Comunicado_Tabela

274 


 2025-11-05 


 Copom mantém a taxa Selic em 15,00% a.a. 


 O ambiente externo ainda se mantém incerto em função da conjuntura e da política econômica nos Estados Unidos, com reflexos nas condições financeiras globais. Tal cenário exige particular cautela por parte de países emergentes em ambiente marcado por tensão geopolítica.

Em relação ao cenário doméstico, o conjunto dos indicadores segue apresentando, conforme esperado, trajetória de moderação no crescimento da atividade econômica, mas o mercado de trabalho ainda mostra dinamismo. Nas divulgações mais recentes, a inflação cheia e as medidas subjacentes apresentaram algum arrefecimento, mas mantiveram-se acima da meta para a inflação.

As expectativas de inflação para 2025 e 2026 apuradas pela pesquisa Focus permanecem em valores acima da meta, situando-se em 4,5% e 4,2%, respectivamente. A projeção de inflação do Copom para o segundo trimestre de 2027, atual horizonte relevante de política monetária, situa-se em 3,3% no 

,0,1,2,3
0,Índice de preços,2025,2026,2º tri 2027
1,IPCA,"4,6","3,6","3,3"
2,IPCA livres,"4,5","3,6","3,2"
3,IPCA administrados,"5,0","3,4","3,5"


In [3]:
def Process_COPOM_Ata(Ata):

    def Aux_COPOM_LimpaHTML(raw_html):
        if not raw_html:
            return "", None
        
        decoded = html.unescape(raw_html)
        decoded = re.sub(r'[\u200b\ufeff\u00a0]', '', decoded)
        soup = BeautifulSoup(decoded, 'html.parser')
        
        # Extrai todas as tabelas como DataFrame
        table_dfs = []
        for table in soup.find_all('table'):
            rows = []
            for tr in table.find_all('tr'):
                cells = [td.get_text(strip=True) for td in tr.find_all(['td', 'th'])]
                if cells:
                    rows.append(cells)
            if rows:
                df_table = pd.DataFrame(rows)
                table_dfs.append(df_table)
            table.decompose()  # Remove a tabela do HTML original
        
        for br in soup.find_all('br'):
            br.replace_with('\n')
        
        for p in soup.find_all('p'):
            p.insert_after('\n\n')
        
        text = soup.get_text()
        text = re.sub(r'\n\s*\n', '\n\n', text)  
        text = re.sub(r'[ \t]+', ' ', text)    
        text = text.strip()
        
        if table_dfs:
            text += "\n\n[Tabela Extraida Do Comunicado:]\n" + "\n\n".join(
                [
                    '\n'.join([
                        '\t'.join([str(cell) if cell is not None else "" for cell in row])
                        for row in df.values.tolist()
                    ])
                    for df in table_dfs
                ]
            )
            return text, table_dfs[0]  # Retorna o texto e a primeira tabela como DataFrame
        else:
            return text, None

    Link_Url = f"https://www.bcb.gov.br/api/servico/sitebcb/copom/atas_detalhes?nro_reuniao={Ata}"
    df = pd.read_json(Link_Url)
    Conteudo_Full = df.iloc[0]

    Reuniao_Raw = Conteudo_Full.iloc[0].get('nroReuniao')
    DataReferencia_Raw = Conteudo_Full.iloc[0].get('dataReferencia')
    Titulo_Raw = Conteudo_Full.iloc[0].get('titulo')
    Comunicado_Raw = Conteudo_Full.iloc[0].get('textoAta')
    
    Ata_Texto, Ata_Tabela = Aux_COPOM_LimpaHTML(Comunicado_Raw)

    return Reuniao_Raw, DataReferencia_Raw, Titulo_Raw, Ata_Texto, Ata_Tabela


Reuniao_Raw, DataReferencia_Raw, Titulo_Raw, Ata_Texto, Ata_Tabela = Process_COPOM_Ata(Ata=274)
print(Reuniao_Raw, '\n\n\n', DataReferencia_Raw, '\n\n\n', Titulo_Raw, '\n\n\n', Ata_Texto)
Ata_Tabela

274 


 2025-11-05 


 274ª Reunião - 4-5 novembro, 2025 


 A) Atualização da conjuntura econômica e do cenário do Copom11. O ambiente externo ainda se mantém incerto em função da conjuntura e da política econômica nos Estados Unidos, com reflexos nas condições financeiras globais. Tal cenário exige particular cautela por parte de países emergentes em ambiente marcado por tensão geopolítica.

2. Em relação ao cenário doméstico, o conjunto dos indicadores segue apresentando, conforme esperado, trajetória de moderação no crescimento da atividade econômica, mas o mercado de trabalho ainda mostra dinamismo.

3. Nas divulgações mais recentes, a inflação cheia e as medidas subjacentes apresentaram algum arrefecimento, mas mantiveram-se acima da meta para a inflação. As expectativas de inflação para 2025 e 2026 apuradas pela pesquisa Focus permanecem em valores acima da meta, situando-se em 4,5% e 4,2%, respectivamente.

B) Cenários e análise de riscos4. O cenário prospectivo de inflação seg

,0,1,2,3
0,Índice de preços,2025,2026,2º tri 2027
1,IPCA,"4,6","3,6","3,3"
2,IPCA livres,"4,5","3,6","3,2"
3,IPCA administrados,"5,0","3,4","3,5"
